# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 12 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.


### Exercise 12.1 

**What are the powers of $11$ when computed$\mod 21$?**

In [1]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Logical;

function Powers(generator : Int, modulus : Int) : Int[] {
    mutable powers = [1];
    mutable acc = generator;
    
    while (acc != 1) {
        set powers += [acc];
        set acc = acc * generator % modulus;
    }
    
    return powers;
}

Powers

In [2]:
%simulate Powers generator=11 modulus=21

1, 11, 16, 8, 4, 2

**How long does it take to loop back around to $11^0 = 1$?**

It takes $11^6 = 1$ before the powers of $11$ loop back around, so we say that the period of the function $f(x) = 11^n \mod 21$ is 6.
That is, $f(x) = f(x + 6)$ for all $x$, and $6$ is the smallest number for which this is the case.

**Does it matter if you take the modulus by $21$ at the end, or whether you compute the modulus at each step?**

No, you can multiply your accumulated value by 11 and then take it mod 21 and then repeat that as many times as needed, and it is the same as modding at the end. The incremental approach scales better as you don't have to store such large numbers and then mod them.

----
### Exercise 12.2

**Using either Python or Q#, try showing that one or both of the potential factor candidates from the last step of Shor's algorithm share factors with the modulus. Use $N = 143$, $g = 19$, and the period $r = 60$.**

Recall step 6 of Shor's Algorithm is:
> 6. Either $g^{\frac{r}{2}} - 1$ or $g^{\frac{r}{2}} + 1$ shares a factor with $N$.

Thus, you'll want to confirm that either $g^{r / 2} - 1 \bmod N$ or $g^{r / 2} + 1 \bmod N$ shares a factor with $N$; that is, isn't coprime with $N$.
In Q#, it's easy to check if two integers are coprime by using the [`IsCoprimeI` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.iscoprimei).

In [3]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

function SharedFactorsFact(generator : Int, modulus : Int, period : Int) : Unit {
    let halfPower = ExpModI(generator, period / 2, modulus);
    Fact(
        not IsCoprimeI(modulus, halfPower + 1 % modulus) or
        not IsCoprimeI(modulus, halfPower - 1 % modulus),
        $"Neither {generator}^({period} / 2) + 1 or {generator}^({period} / 2) - 1 shares a factor with {modulus}."
    );
    
}

SharedFactorsFact

In [4]:
%simulate SharedFactorsFact generator=19 modulus=143 period=60

()

----
### Exercise 12.3

**What's the GCD of 35 and 30?**

In [5]:
open Microsoft.Quantum.Math;
function ExerciseGCD() : Int {
    return GreatestCommonDivisorI (35, 30); 
}

ExerciseGCD

In [6]:
%simulate ExerciseGCD

5

This does help you find the factors of $35$ because now you know that $5$ is a factor in common between $35$ and $30$ and thus is also a factor of $35$. You can then divide $35$ by $5$ and get that $7$ is also a factor of $35$.

----
### Exercise 12.4

**What state would your register be in after multiplying by 5 modulo 9?**

> Note: the code below is the same sample code that is in `IntegerFactorization.ipynb`.

In [7]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;

operation PrepareSuperpositionOfTwoInts(register : LittleEndian, intPair : (Int, Int)) 
: Unit is Adj + Ctl {
    use ctrl = Qubit();
    H(ctrl);
    within{
        X(ctrl);
    } 
    apply{
        Controlled ApplyXorInPlace([ctrl], (Fst(intPair), register));
    }

    Controlled ApplyXorInPlace([ctrl], (Snd(intPair), register));
    (ControlledOnInt(Snd(intPair), Y))(register!, ctrl);  
}


operation MultiplyInSuperpositionMod(superpositionInt1 : Int, 
    superpositionInt2 : Int, multiplier : Int, modulus : Int) 
: Int {
    use target = Qubit[BitSizeI(modulus)];

    PrepareSuperpositionOfTwoInts(LittleEndian(target), (superpositionInt1, superpositionInt2));
        
    Message("Before multiplication:");
    DumpMachine();
        
    MultiplyByModularInteger(multiplier, modulus, LittleEndian(target));
        
    Message("After multiplication:");
    DumpMachine();
    return MeasureInteger(LittleEndian(target));
}

PrepareSuperpositionOfTwoInts, MultiplyInSuperpositionMod

In [8]:
%simulate MultiplyInSuperpositionMod superpositionInt1=2 superpositionInt2=7 multiplier=5 modulus=9

Before multiplication:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + -0.7071067811865476𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖

After multiplication:


|0⟩	8.048776560288357E-33 + -1.460122887337483E-32𝑖
|1⟩	0.7071067811865606 + 1.9992185179568873E-16𝑖
|2⟩	-8.65436685860561E-33 + 3.4429570232036055E-33𝑖
|3⟩	-6.577780649231735E-33 + -2.102246182297412E-32𝑖
|4⟩	2.99216292339912E-32 + -5.787841102164402E-32𝑖
|5⟩	-1.7283263829171187E-32 + -1.620050236154245E-32𝑖
|6⟩	3.9739786835464645E-32 + 3.776080382166454E-32𝑖
|7⟩	1.3396819126055798E-32 + -1.358643685327334E-33𝑖
|8⟩	1.6581641771351613E-16 + -0.707106781186564𝑖
|9⟩	6.733339744291133E-33 + -4.237286353411284E-33𝑖
|10⟩	9.12850440141019E-18 + 4.600413928980151E-17𝑖
|11⟩	-2.1154384951130604E-33 + -1.2052339377170018E-32𝑖
|12⟩	7.690881461009681E-33 + 6.904054068179995E-33𝑖
|13⟩	-9.59407778011189E-33 + 1.2559367772680923E-32𝑖
|14⟩	-3.4477542521925934E-33 + -5.183290758564699E-33𝑖
|15⟩	3.074280240429881E-33 + -8.709147993958713E-33𝑖

8

You can see that after multiplication, the state is $\frac{1}{\sqrt{2}}\left(\left|1\right\rangle + \left|8\right\rangle\right)$.

----
### Exercise 12.5

**Try to multiply by 3 modulo 9, you'll get an error; why?**

In [9]:
%simulate MultiplyInSuperpositionMod superpositionInt1=2 superpositionInt2=7 multiplier=3 modulus=9

Before multiplication:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + -0.7071067811865476𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖

Source,Callable
D:\a\1\s\submodules\QuantumLibraries\Standard\src\Arithmetic\Modular.qs:234,Microsoft.Quantum.Arithmetic.MultiplyByModularInteger
(notebook),MultiplyInSuperpositionMod


`constMultiplier` and `modulus` must be co-prime
	Expected:	True
	Actual:	False


<details>
    <summary>Explanation</summary>
    Multiplying by $3 \mod 9$ is not reversible.
    For instance, both $1 \times 3$ and $4 \times 3$ mod 9 give 0, even though $1 \ne 4 \mod 9$.
    Since classical functions have to be reversible in order to be represented by quantum operations, the <code>MultiplyByModularInteger</code> operation raises an error in this case.
</details>
   

----
### Epilogue

_The following cell logs what version of the components this was last tested with._

In [ ]:
%version